In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive/')

In [ ]:
base_dir = '/content/drive/MyDrive/Colab Notebooks/'
!ls "/content/drive/My Drive/Colab Notebooks"

In [ ]:
#menentukan direktori
bahan_dir = os.path.join(base_dir, 'bahan')
train_dir = os.path.join(base_dir, 'latih')
validation_dir = os.path.join(base_dir, 'validasi')

In [ ]:
#menentukan direktory isi bahan
pandan_dir = os.path.join(bahan_dir, 'pandan/')
sirih_dir = os.path.join(bahan_dir, 'sirih/')

print("Jumlah data Train Tiap Kelas")
print('Jumlah gambar daun pandan :',len(os.listdir(pandan_dir)))
print('Jumlah gambar daun sirih :',len(os.listdir(sirih_dir)))

Jumlah data Train Tiap Kelas
Jumlah gambar daun pandan : 50
Jumlah gambar daun sirih : 50


In [ ]:
#direktori isi latihan/training
train_pandan =os.path.join(train_dir, 'pandan/')
train_sirih = os.path.join(train_dir, 'sirih/')

#direktori isi validasi
validation_pandan = os.path.join(validation_dir, 'pandan/')
validation_sirih = os.path.join(validation_dir, 'sirih/')

menyiapkan data

In [ ]:
import random
from shutil import copyfile

def train_val_split(source, train, val, train_ratio):
  total_size = len (os.listdir(source))
  train_size = int (train_ratio * total_size)
  val_size = total_size - train_size

  randomized = random.simple(os.listdir(source), total_size)
  train_files = randomized[0:train_size]
  val_files = randomized[train_size:total_size]

  for i in train_files:
    i_file = source + i
    destination = train + i
    copyfile(i_file, destination)

  for i in val_files:
    i_file = source + i
    destination = val + i
    copyfile(i_file, destination)

#jumlah pembagian data training dan testing
train_ratio = 0.9

#pembagian training dan validasi 
#training
source_00 = pandan_dir
train_00 = train_pandan
val_00 = validation_pandan
train_val_split(source_00, train_00, val_00, train_ratio)

#validasi
source_01 = sirih_dir
train_01 = train_sirih
val_01 = validation_sirih
train_val_split(source_01, train_01, val_01, train_ratio)

AttributeError: ignored

In [ ]:
print('Jumlah All daun pandan   :', len(os.listdir(pandan_dir)))
print('Jumlah Train daun pandan   :', len(os.listdir(train_pandan)))
print('Jumlah val  daun pandan   :', len(os.listdir(validation_pandan)))

pre procesing

In [ ]:
import tensorflow as tf
from transorflow.keras.optimizers import adam
from transorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    ratation_range = 30,
    horizontal_flip = True
    shear_range = 0.3,
    fill_mode = 'nearest',
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range =0.1
)

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    ratation_range = 30,
    horizontal_flip = True
    shear_range = 0.3,
    fill_mode = 'nearest',
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range =0.1
)

target

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size = (150, 150),
    batch_size = 10,
    class_mode = 'categorical'
)

In [ ]:
class myCallback(tf.keras.callbacks.callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.99):
      print('\nAkurasi mencapai 99%')
      self.model.stop_training = True
      
callbacks = myCallback()

Model CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3))
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.conv2D(32, (3, 3), activation = 'rule'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.conv2D(64,(3, 3), activation = 'rule')
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(200, activation = 'rule'),
    tf.keras.layers.Dropout(0.3, seed=112),
    tf.keras.layers.Dense(500, activation = 'rule'),
    tf.keras.layers.Dropout(0.5,seed=112),
    tf.keras.layers.Dense(2, activevation = 'sigmoid')
])

In [ ]:
model.summary

In [ ]:
model.compile(loss = 'categorycal_croddentropy',
              optimized = 'Adam',
              metrics =['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=6,
    epochs = 25,
    validation_data =val_generator,
    validation_steps = 1,
    verbose = 1,
    callbacks = [callbacks]
)

tingkat akurasi dan loss model

In [ ]:
from prompt_toolkit.shortcuts.progress_bar.formatters import Label
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'b', Label = 'validation Accuracy')
plt.tittle('Training and Validasi accurasy')
plt.legend(loc = 'best')
plt.show()


mencoba klasifikasi

In [ ]:
import numpy as np
from keras_preprocessing import image
from google.colab import files

uploaded = files.upload()

for fn in uploaded.key():
  #predicting image
  path = fn
  img = image.load_image(path, target_size =(150, 150))
  imgplot = plt.imgshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)

  images = np.vstack([x])
  classes = model.predict(images, bacth_size =100)

  print(fn)

  class_list = os.listdir(train_dir)

  for j in range(42):
    if classes[0][j] == 1. :
      print('This image belongs to class', class_list[j])
      break